## 7.2 seq2seq
- 시계열 데이터를 다른 시계열 데잍로 변환하는 모델

#### 7.2.1 seq2seq 원리 
- seq2seq를 Encoder-Decoder 모델이라고도 한다.
- Encoder : 입력데이터를 인코딩(부호화)한다.
- Decoder : 인코딩된 데이터를 디코딩(복호화)한다.

![image](https://github.com/choibigo/Study/assets/38881179/eb2dc875-931f-4f4e-a2ba-aa54878336f8)

- "나는 고양이로소이다"를 "I am a cat"으로 번역하는 Task를 수행가능하다.
- 기존 언어 모델은 사용한 문장 생성은 학습 데이터(corpus)에 있는 단어들의 조합으로만 문장을 생성할 수 있었지만, Encoder-Decoder 형태를 사용하면 다른 도메인의 언어를 생성가능하다.
- 입력 문장을 Encoding 하고 이어서 그 인코딩한 정보를 Decoder에 전달하고 Decoder가 최종 문장을 생성한다.
- Encoder와 Decoder로 RNN을 사용할 수 있다.

![image](https://github.com/choibigo/Study/assets/38881179/c47823c7-62c3-44b9-8f9b-c085118cd447)

- Encoder는 RNN을 이용하여 시계열 데이터를 h라는 은닉 상태 벡터로 변환한다.
- 여기에서는 각 LSTM의 output이 중요한 것이 아닌 최종 은닉 상태 벡터가 중요하다.
- 이 은닉 벡터가 Decoder의 첫 입력으로 주어진다.
- 이떄 LSTM의 은닉 상태 h는 고정길이 벡터이다, 따라서 어떤 입력이 주어지더라도 고정된 길이의 벡터가 생성된다.

![image](https://github.com/choibigo/Study/assets/38881179/d0a7069a-3d4a-4569-8dfb-70d6ba65f58c)

- Decoder는 기존 LSTM 사용한 신경망과 동일하다.
- 그러나 첫 입력이 학습데이터가 아닌 Encoder에서 생성된 hidden state라는 것이다.
- 이러한 차이가 언어모델을 번역 모델로 바꿀수 있다.

![image](https://github.com/choibigo/Study/assets/38881179/6457a006-4ea5-469d-afe3-2d73e4c2a0da)

- seq2seq는 두개의 LSTM 계층(Encoder, Decoder)로 구성된다.
- Hidden Satate가 Encoder와 Decoder를 연결하는 다리가 된다.

#### 7.2.2 시계열 데이터 변환용 Simple 문제

![image](https://github.com/choibigo/Study/assets/38881179/8122b87f-30ef-405e-bf7c-274c2f331d5f)

- 입력이 덧셈 식으로 주어지고 출력이 그 연산의 답으로 나오는 신경망을 구성할 수 있다.
- seq2seq는 덧셈의 예제로 부터 문자의 패턴을 학습하고 덧셈의 규칙을 올바르게 학습 해야 한다.
- word2vec이나 언어 모델에서는 문장을 '단어' 단위로 분할해 왔지만 반드시 단어 단위로 분할해야 하는 것은 아니다.
- 이번 문제에서는 단어가 아닌 '문자'단위로 분할한다.

#### 7.2.3 가변길이 시계열 데이터
- 입력이 "57+5", "628+521"가 주어진다면 4문자, 7문자로 입력의 길이가 모두 다르다.
- 이처럼 샘플마다 데이터의 시간 방향 크기가 다르다. 앞선 예제들에서는 T 크기로 자른 입력들이 각 LSTM 계층으로 입력으로 주어졌으므로 입력 데이터 크기에 대해서는 고려하지 않아도 됬다.
- 크기가 다르기 때문에 미니 배치로 학습이 불가능 하기 때문에 Padding을 사용할 수 있다.


![image](https://github.com/choibigo/Study/assets/38881179/67835cf4-9bac-46ff-ab05-b38b4110b146)

- padding을 사용하여 가변적인 데이터의 크기를 고정 크기로 만들 수 있다.
- 추가적으로 질문과 정답을 구분하기 위해서 출력 앞에는 '_'를 붙이기로 한다.
- 그 결과 출력데이터의 최대 길이는 5 문자가 된다, 이 구분자는 Decoder에 문자열을 생성하라고 알리는 신호로 사용된다.
- 이처럼 Padding을 이용하여 데이터 크기를 통일 시키면 가변 길이 시게열 데이터도 처리 가능하다.
- Padding을 사용한다면 seq2seq에 패딩 전용 처리를 추가 해야한다, Decoder에 입력된 데이터가 패딩이라면 손실의 결과에 반영하지 않도록 해야한다.
- softmax와 loss 함수에 마스크를 추가해 해결할 수 있다.
- Encoder에 입력된 데이터가 패딩이라면 LSTM 계층이 이전 시각의 입력을 그대로 출력하게 한다, LSTM 계층은 처음부터 패딩이 존재하지 않았던 것처럼 인코딩할 수 있다.


#### 7.2.4 덧셈 데이터 셋
```txt
16+75  _91  
52+607 _659 
75+22  _97  
63+22  _85  
795+3  _798 
706+796_1502
...
621+0  _621 
81+89  _170 
846+84 _930 
27+63  _90  
4+582  _586 
7+1    _8   
63+5   _68  
334+0  _334 
```

In [1]:
import sys
sys.path.append('..')
from data_set import sequence

(x_train, t_train), (x_test, t_test) = sequence.load_data(r'D:\workspace\Difficult\my_repository\Deep Learning 스터디\밑바닥 부터 시작하는 딥러닝 2\CH7\addition.txt')
char_to_id, id_to_char = sequence.get_vocab() # 각 문자의 idx를 담고있는 정보

print(x_train.shape) # Padding된 문장에서 각 문자를 id값으로 치환한 데이터
print(t_train.shape)

(45000, 7)
(45000, 5)


## 7.3 seq2seq 구현
- seq2seq는 2개의 RNN을 연결한 신경망이다.

#### 7.3.1 Encoder 클래스 구현
![image](https://github.com/choibigo/Study/assets/38881179/15464c83-3fc5-46fc-aca0-a8be05996a43)

- Encoder는 문자열을 받아 벡터 h로 변환한다.
- LSTM 계층을 이용하여 구현할 수 있다.

![image](https://github.com/choibigo/Study/assets/38881179/5b1c34fc-33aa-4b56-a97e-450e150552bc)

- Encoder 클래스는 Embedding 계층과 LSTM 계층으로 구성된다.
- Embedding 계층에서는 문자 ID를 문자 벡터로 변환한다.
- LSTM 계층은 오른쪽으로는 은닉상태와 셀을 출력하고 위쪽으로는 은닉상태만을 출력한다.
- seq2seq에서는 위쪽 출력을 필요 없으므로 폐기 한다.
- Encoder에서는 마지막 문자를 처리한 후 LSTM 계층의 은닉 상태 h를 출력한다, 이후 h가 Decoder로 전달 된다.
- LSTM의 cell도 Decoder에 전달할 수 있지만, LSTM의 셀을 다른 계층에 전달하는 일은 일반적으로 흔치 않다, LSTM cell 의 설계가 자기 자신만 사용하도록 되어있기 때문이다.

![image](https://github.com/choibigo/Study/assets/38881179/bdb7cdd8-2338-49e8-bac7-369a9c8ef61c)

In [2]:
import numpy as np
from numpy.random import randn as rn
from common.time_layers import TimeEmbedding, TimeLSTM, TimeAffine

class Encoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size

        embed_W = (rn(V, D)/100).astype(np.float64)
        lstm_Wx = (rn(D, 4 * H)/100).astype(np.float64)
        lstm_Wh = (rn(H, 4 * H)/100).astype(np.float64)
        lstm_b = np.zeros(4*H).astype(np.float64)

        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=False)
        self.params = self.embed.params + self.lstm.params
        self.grads = self.embed.grads + self.lstm.grads
        self.hs = None
    
    def forward(self, xs):
        xs = self.embed.forward(xs)
        hs = self.lstm.forward(xs)
        self.hs = hs
        return hs[: -1, :] # 마지막 hidden state만 반환한다.
    
    def backward(self, dh):
        dhs = np.zeros_like(self.hs)
        dhs[:, -1, :] = dh # 마지막 hidden state의 기울기만 뽑아 낸다.
        
        dout = self.lstm.backward(dhs)
        dout = self.embed.backward(dout)
        return dout

#### 7.3.2 Decoder 클래스

![image](https://github.com/choibigo/Study/assets/38881179/67130fba-99bd-4e88-ae73-c17f7deb3bc7)

- Encoder 클래스가 출력한 h를 입력 받아 Target Domain의 문자열로 출력한다.
- Decoder 또한 RNN으로 구현할 수 있다.

![image](https://github.com/choibigo/Study/assets/38881179/4a6d0c3e-640d-4290-b590-c6613d4a3bae)

- 정답 데이터는 "_62"로 정의되어 있지만, 입력데이터를 ```['_','6','2','']```으로 주고, 이에 대응하는 ```['6','2','','']```이 되도록 학습 시킨다.
```
- RNN으로 문장을 학습할때와 추론할때는 데이터를 부여하는 방법이 다르다.
- 학습 시에는 모든 문장을 알기 때문에 LSTM 계층의 입력을 이전 출력과 관계없이 정답을 부여할 수 있다.
- 학습시에는 시계열 방향의 데이터를 한꺼번에 줄 수 있다.
- 추론시에는 문장전체를 모르기 때문에 최초 시작을 알리는 구분 문자(이번 예시에서는 '_') 하나를 부여 한다.
```


![image](https://github.com/choibigo/Study/assets/38881179/432a8bac-6dad-4e88-bc18-e71a171bfd25)

- 이번 예시에서는 '확률적'결정 방법이 아닌 '결정적'인 결정 방법으로 최종 출력을 결정한다.
- argmax라는 노드를 통해서 최대 값을 가진 index를 반환한다. (Decoder 계층 에서는 Score의 최대만 알면 되기 때문에 Softmax 함수는 필요 없다.)

![image](https://github.com/choibigo/Study/assets/38881179/05e3ab83-2df1-4667-a38f-17af8bdc518b)
- Decoder의 결과(결정된 ID)와 정답 값을 softmax에 입력으로 준뒤 Loss를 계산한다.
- Decoder는 Time Embedding과 Time LSTM, Time Afiine 3개의 계층으로 구성된다.

In [3]:

class Decoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        
        embed_W = (rn(V, D)/100).astype(np.float64)
        lstm_Wx = (rn(D, 4 * H)/100).astype(np.float64)
        lstm_Wh = (rn(H, 4 * H)/100).astype(np.float64)
        lstm_b = np.zeros(4*H).astype(np.float64)
        affine_W = (rn(H, V) / np.sqrt(H)).astype(np.float64)
        affine_b = np.zeros(V).astype(np.float64)

        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=False)
        self.affine = TimeAffine(affine_W, affine_b)

        self.params, self.grads = [], []
        for layer in (self.embed, self.lstm, self.affine):
            self.params += layer.params
            self.grads += layer.grads
        
    def forward(self, xs, h):
        self.lstm.set_state(h) # Encoder의 출력인 Hidden State를 LSTM의 hidden state로 세팅한다.

        out = self.embed.forward(xs)
        out = self.lstm.forward(out)
        score = self.affine.forward(out)
        return score

    def backward(self, dscore):
        dout = self.affine.backward(dscore)
        dout = self.lstm.backward(dout)
        dout = self.embed.backward(dout)
        dh = self.lstm.dh # Ebcider로 흘려 보내줄 lstm의 hidden state의 grads를 따로 변수에 저장 후 반환

        return dh

    def generate(self, h, start_id, sample_size):
        '''
        - 학습 시에는 고정된 길이의 데이터가 출력으로 반환된다.
        - 실제 문장을 생성할 때는 sample_size가 주어지고 그 크기 만큼 데이터가 생성된다.
        '''

        sampled = []
        sample_id = start_id
        self.lstm.set_state(h)

        for _ in range(sample_size):
            x = np.array(sample_id).reshape((1, 1)) # network가 batch 처리를 가능하도록 설계 되었기 때문에 입력을 1x1로 만들어 준다.
            out = self.embed.forward(x)
            out = self.lstm.forward(out)
            score = self.affine.forward(out)

            sample_id = np.argmax(score.flatten()) # 1x1 형태로 출력되기 때문에 형태를 1차원으로 바꿔준다.
            sampled.append(int(sample_id))
        
        return sampled

#### 7.3.3 seq2seq 클래스 
- Encoder 클래스와 Decoder 클래스를 연결하고 softmax 와 loss를 이용해 손실을 계산한다.

In [4]:
import sys
sys.path.append('..')

from common.base_model import BaseModel
from common.time_layers import TimeSoftmaxWithLoss

class Seq2seq(BaseModel):
    def __init__(self, bocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        self.encoder = Encoder(V, D, H)
        self.decoder = Decoder(V, D, H)
        self.softmax = TimeSoftmaxWithLoss()

        # parameter와 grad를 모음
        self.params = self.encoder.params + self.decoder.params
        self.grads = self.encoder.grads + self.decoder.grads

    def forward(self, xs, ts):
        decoder_xs, decoder_ts = ts[:, :-1], ts[:, 1:] # decoder의 입력과 정답
        h = self.encoder.forward(xs)
        score = self.decoder.forward(decoder_xs, h)
        loss = self.softmax.forward(score, decoder_ts)
        return loss

    def backward(self, dout=1):
        dout = self.softmax.backward(dout)
        dh = self.decoder.backward(dout)
        dout = self.encoder.backward(dh)
        return dout

    def generate(self, xs, start_id, sample_size):
        h = self.encoder.forward(xs)
        sampled = self.decoder.generate(h, start_id, sample_size)
        return sampled

#### 7.3.4 seq2seq 평가

In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
from data_set import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
# from peeky_seq2seq import PeekySeq2seq


# 데이터셋 읽기
(x_train, t_train), (x_test, t_test) = sequence.load_data('addition.txt')
char_to_id, id_to_char = sequence.get_vocab()

# 입력 반전 여부 설정 =============================================
is_reverse = False  # True
if is_reverse:
    x_train, x_test = x_train[:, ::-1], x_test[:, ::-1]
# ================================================================

# 하이퍼파라미터 설정
vocab_size = len(char_to_id)
wordvec_size = 16
hidden_size = 128
batch_size = 128
max_epoch = 50

max_grad = 5.0

# 일반 혹은 엿보기(Peeky) 설정 =====================================
model = Seq2seq(vocab_size, wordvec_size, hidden_size)
# model = PeekySeq2seq(vocab_size, wordvec_size, hidden_size)
# ================================================================
optimizer = Adam()
trainer = Trainer(model, optimizer)

acc_list = []
for epoch in range(max_epoch):
    trainer.fit(x_train, t_train, max_epoch=1,
                batch_size=batch_size, max_grad=max_grad)

    correct_num = 0
    for i in range(len(x_test)):
        question, correct = x_test[[i]], t_test[[i]]
        verbose = i < 10
        correct_num += eval_seq2seq(model, question, correct,
                                    id_to_char, verbose, is_reverse)

    acc = float(correct_num) / len(x_test)
    acc_list.append(acc)
    print('검증 정확도 %.3f%%' % (acc * 100))

# 그래프 그리기
x = np.arange(len(acc_list))
plt.plot(x, acc_list, marker='o')
plt.xlabel('에폭')
plt.ylabel('정확도')
plt.ylim(0, 1.0)
plt.show()